In [1]:
import collections
import tensorflow as tf
import numpy as np

In [1]:
https://github.com/google/sentencepiece
tf.split
tf.contrib.framework.nest.
tf.contrib.lookup.index_table_from_file
tf.gfile.Glob
tf.data.TextLineDataset

tf.data.Dataset.zip
tf.data.Dataset.shard
tf.data.Dataset.shuffle
tf.data.Dataset.map
tf.data.Dataset.filter
tf.data.Dataset.from_tensor_slices
tf.contrib.seq2seq.tile_batch
tf.contrib.rnn.BasicLSTMCell

tf.string.split
tf.slice
tf.layer.Dense
tf.nn.dymatic.rnn
tf.nn.bidirection_dymatic_rnn

tf.contrib.seq2seq.BaseDecoder
tf.contrib.seq2seq.BeamSearchDecoder
tf.contrib.seq2seq.BeamSearchDecoderOutput
tf.contrib.seq2seq.BeamSearchDecoderState


tf.contrib.seq2seq.TrainHelper
tf.contrib.seq2seq.GreedEmbeddingHelper
tf.contrib.seq2seq.SampleEmbeddingHelper
tf.contrib.seq2seq.dynamic_decoder

tf.contrib.rnn.BasicLSTMCell
tf.contrib.rnn.DropoutWrapper
tf.contrib.rnn.MultiRNNCell

tf.Summary
tf.Summary.Value

SyntaxError: invalid syntax (<ipython-input-1-90fa55a1f917>, line 1)

In [3]:
class BatchedInput(
    collections.namedtuple("BatchedInput",
                           ("initializer", "source", "target_input",
                            "target_output", "source_sequence_length",
                            "target_sequence_length"))):
  pass

In [4]:
def get_iterator(src_dataset,
                 tgt_dataset,
                 src_vocab_table,
                 tgt_vocab_table,
                 batch_size,
                 sos,
                 eos,
                 random_seed,
                 num_buckets,
                 src_max_len=None,
                 tgt_max_len=None,
                 num_parallel_calls=4,
                 output_buffer_size=None,
                 skip_count=None,
                 num_shards=1,
                 shard_index=0,
                 reshuffle_each_iteration=True,
                 use_char_encode=False):
  output_buffer_size = batch_size * 1000


  src_tgt_dataset = tf.data.Dataset.zip((src_dataset, tgt_dataset))

  tgt_sos_id='0'
  tgt_eos_id='1'
  src_sos_id='0'
  src_eos_id='1'

 #把一行转化成一个个单词序列
  src_tgt_dataset = src_tgt_dataset.map(
      lambda src, tgt: (
          tf.string_split([src]).values, tf.string_split([tgt]).values),
      num_parallel_calls=num_parallel_calls).prefetch(output_buffer_size)

  # Filter zero length input sequences.
  src_tgt_dataset = src_tgt_dataset.filter(
      lambda src, tgt: tf.logical_and(tf.size(src) > 0, tf.size(tgt) > 0))


  # Convert the word strings to ids.  Word strings that are not in the

#   src_tgt_dataset = src_tgt_dataset.map(
#         lambda src, tgt: (tf.cast(src_vocab_table.lookup(src), tf.int32),
#                           tf.cast(tgt_vocab_table.lookup(tgt), tf.int32)),
#         num_parallel_calls=num_parallel_calls)
  #src_tgt_dataset是单词序列,序列值是单词索引
  src_tgt_dataset = src_tgt_dataset.prefetch(output_buffer_size)
  # Create a tgt_input prefixed with <sos> and a tgt_output suffixed with <eos>.
  #对目标头为加入了tgt_sos_id,tgt_eos_id,(src,tgt)->(src,<sos>+tgt,tgt+<eos>)
  src_tgt_dataset = src_tgt_dataset.map(
      lambda src, tgt: (src,
                        tf.concat(([tgt_sos_id], tgt), 0),
                        tf.concat((tgt, [tgt_eos_id]), 0)),
      num_parallel_calls=num_parallel_calls).prefetch(output_buffer_size)
  # Add in sequence lengths.

  src_tgt_dataset = src_tgt_dataset.map(
        lambda src, tgt_in, tgt_out: (
            src, tgt_in, tgt_out, tf.size(src), tf.size(tgt_in)),
        num_parallel_calls=num_parallel_calls)
  #src_tgt_dataset 表示(src,tgt_in,tgt_out,src.size,tgt_in.size)
  src_tgt_dataset = src_tgt_dataset.prefetch(output_buffer_size)

  # Bucket by source sequence length (buckets for lengths 0-9, 10-19, ...)
  '''
  x是一个(src,tgtin,tgtout,size(src),size(tgt))的数据集,
  返回批处理的数据库,shape=(N,) 每个元素是一个(src,tgtin,tgtout,size(src),size(tgt))
  由于每一行长度不同(src,tgin,tgout),会做pad处理,pad_value分别是src_eos_id,tgt_eos_id,tgt_eos_id
  '''
  def batching_func(x):
    return x.padded_batch(
        batch_size,
        # The first three entries are the source and target line rows;
        # these have unknown-length vectors.  The last two entries are
        # the source and target row sizes; these are scalars.
        padded_shapes=(
            tf.TensorShape([None]),  # src
            tf.TensorShape([None]),  # tgt_input
            tf.TensorShape([None]),  # tgt_output
            tf.TensorShape([]),  # src_len
            tf.TensorShape([])),  # tgt_len
        # Pad the source and target sequences with eos tokens.
        # (Though notice we don't generally need to do this since
        # later on we will be masking out calculations past the true sequence.
        padding_values=(
            src_eos_id,  # src
            tgt_eos_id,  # tgt_input
            tgt_eos_id,  # tgt_output
            0,  # src_len -- unused
            0))  # tgt_len -- unused


  batched_dataset = batching_func(src_tgt_dataset)
  batched_iter = batched_dataset.make_initializable_iterator()
  (src_ids, tgt_input_ids, tgt_output_ids, src_seq_len,
   tgt_seq_len) = (batched_iter.get_next())
  return BatchedInput(
      initializer=batched_iter.initializer,
      source=src_ids,
      target_input=tgt_input_ids,
      target_output=tgt_output_ids,
      source_sequence_length=src_seq_len,
      target_sequence_length=tgt_seq_len)

In [10]:
src_dataset=tf.data.TextLineDataset('/home/zhangxk/AIProject/nmt/nmt/scripts/iwsl515/train.en')
tgt_dataset=tf.data.TextLineDataset('/home/zhangxk/AIProject/nmt/nmt/scripts/iwsl515/train.vi')
src_vocab_table=tf.contrib.lookup.index_table_from_file('/home/zhangxk/AIProject/nmt/nmt/scripts/iwsl515/vocab.en')
tgt_vocab_table=tf.contrib.lookup.index_table_from_file('/home/zhangxk/AIProject/nmt/nmt/scripts/iwsl515/vocab.vi')
batch_size=9
sos,eos='<sos>','<eos>'

In [11]:
batchinput=get_iterator(src_dataset,
                 tgt_dataset,
                 src_vocab_table,
                 tgt_vocab_table,
                 batch_size,
                 sos,
                 eos,
                 random_seed=0,
                 num_buckets=1,
                 num_shards=1,
                 shard_index=0,
                 reshuffle_each_iteration=False,
                 use_char_encode=False)

In [12]:
print(batchinput.source)
print(batchinput.target_input)
print(batchinput.source_sequence_length)

Tensor("IteratorGetNext_1:0", shape=(?, ?), dtype=string)
Tensor("IteratorGetNext_1:1", shape=(?, ?), dtype=string)
Tensor("IteratorGetNext_1:3", shape=(?,), dtype=int32)


In [13]:
with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    sess.run(batchinput.initializer)
    for i in range(2):
        x=sess.run([batchinput.source,
                     batchinput.target_input,
                     batchinput.target_output,
                    batchinput.source_sequence_length,
                    batchinput.target_sequence_length])
        print(x[0].shape)
        print(x[1].shape)
        print(x[2].shape)
        print(x[3])
        print(x[4])
        print('xxxxxxxxx')

(9, 50)
(9, 77)
(9, 77)
[ 9 50 27 31 13 31 11 10 18]
[11 77 31 31 19 39 17 11 25]
xxxxxxxxx
(9, 44)
(9, 62)
(9, 62)
[29 12 23 44 43 30 22 29 17]
[37 19 26 62 51 35 21 32 17]
xxxxxxxxx


In [9]:
print(x[0].shape)
print(x[1].shape)
print(x[2].shape)
print(x[3])
print(x[4])

(3, 31)
(3, 39)
(3, 39)
[31 13 31]
[31 19 39]


In [2]:
tf.reset_default_graph()
BATCH,T,D=22,10,64
X=tf.placeholder(dtype=tf.float32,shape=[BATCH,T,D])
with tf.variable_scope('my',tf.AUTO_REUSE):
    ndims=[128,256,512]
    cells=[tf.contrib.rnn.BasicLSTMCell(d) for d in ndims]
#state_is_tuple,有多少成返回的state是个tuple,就有多少个元素,每个元素又是LSTMStateTuple,保存lstm的s,h,都是tensor类型
    complete_cell=tf.contrib.rnn.MultiRNNCell(cells,state_is_tuple=True)
    zero_state=complete_cell.zero_state(BATCH,dtype=tf.float32)
    states=zero_state
    outputs=[]
    for t in range(T):
        output,states=complete_cell(X[:,t,:],states)
        outputs.append(output)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


(LSTMStateTuple(c=<tf.Tensor 'my/MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(22, 128) dtype=float32>, h=<tf.Tensor 'my/MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(22, 128) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'my/MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(22, 256) dtype=float32>, h=<tf.Tensor 'my/MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(22, 256) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'my/MultiRNNCellZeroState/BasicLSTMCellZeroState_2/zeros:0' shape=(22, 512) dtype=float32>, h=<tf.Tensor 'my/MultiRNNCellZeroState/BasicLSTMCellZeroState_2/zeros_1:0' shape=(22, 512) dtype=float32>))


In [3]:
print(outputs) #output是最高层的输出,shape(N,D_last_layer)
print(len(outputs))
print(states) #(outputs,(layer1_state,layer2_state,layer3_state))
'''
#总结:对于MultiRNNCell单元,

输入:
    X:[N,T]
    state:(l1_state,l2_state,...ln_state)
    li_state:LSTMStateTuple(c:Tensor,h:Tensor)
输出
    output:(N,ln_ndims),最后一次的h
    state:(l1_state,l2_state,...ln_state),li_state:LSTMStateTuple(c:Tensor,h:Tensor)
'''

[<tf.Tensor 'my/my/multi_rnn_cell/cell_2/basic_lstm_cell/Mul_2:0' shape=(22, 512) dtype=float32>, <tf.Tensor 'my/my/multi_rnn_cell/cell_2/basic_lstm_cell/Mul_5:0' shape=(22, 512) dtype=float32>, <tf.Tensor 'my/my/multi_rnn_cell/cell_2/basic_lstm_cell/Mul_8:0' shape=(22, 512) dtype=float32>, <tf.Tensor 'my/my/multi_rnn_cell/cell_2/basic_lstm_cell/Mul_11:0' shape=(22, 512) dtype=float32>, <tf.Tensor 'my/my/multi_rnn_cell/cell_2/basic_lstm_cell/Mul_14:0' shape=(22, 512) dtype=float32>, <tf.Tensor 'my/my/multi_rnn_cell/cell_2/basic_lstm_cell/Mul_17:0' shape=(22, 512) dtype=float32>, <tf.Tensor 'my/my/multi_rnn_cell/cell_2/basic_lstm_cell/Mul_20:0' shape=(22, 512) dtype=float32>, <tf.Tensor 'my/my/multi_rnn_cell/cell_2/basic_lstm_cell/Mul_23:0' shape=(22, 512) dtype=float32>, <tf.Tensor 'my/my/multi_rnn_cell/cell_2/basic_lstm_cell/Mul_26:0' shape=(22, 512) dtype=float32>, <tf.Tensor 'my/my/multi_rnn_cell/cell_2/basic_lstm_cell/Mul_29:0' shape=(22, 512) dtype=float32>]
10
(LSTMStateTuple(c=<

'\n#总结:对于MultiRNNCell单元,\n\n输入:\n    X:[N,T]\n    state:(l1_state,l2_state,...ln_state)\n    li_state:LSTMStateTuple(c:Tensor,h:Tensor)\n输出\n    output:(N,ln_ndims),最后一次的h\n    state:(l1_state,l2_state,...ln_state),li_state:LSTMStateTuple(c:Tensor,h:Tensor)\n'

In [4]:
tf.reset_default_graph()
BATCH,T,D=22,10,64
X=tf.placeholder(dtype=tf.float32,shape=[BATCH,T,D])
with tf.variable_scope('mMmM',reuse=tf.AUTO_REUSE):
    ndims=[32,64]
    cells=[tf.contrib.rnn.BasicLSTMCell(d) for d in ndims]
    complete_cell=tf.contrib.rnn.MultiRNNCell(cells)
    
    output,states=tf.nn.dynamic_rnn(complete_cell,X,
                      sequence_length=None,
                      initial_state=None,
                      dtype=tf.float32,
                      swap_memory=True,
                      time_major=False)

In [5]:
print(output)
print(states)

Tensor("mMmM/rnn/transpose_1:0", shape=(22, 10, 64), dtype=float32)
(LSTMStateTuple(c=<tf.Tensor 'mMmM/rnn/while/Exit_3:0' shape=(22, 32) dtype=float32>, h=<tf.Tensor 'mMmM/rnn/while/Exit_4:0' shape=(22, 32) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'mMmM/rnn/while/Exit_5:0' shape=(22, 64) dtype=float32>, h=<tf.Tensor 'mMmM/rnn/while/Exit_6:0' shape=(22, 64) dtype=float32>))


In [6]:
#bi_direction rnn
tf.reset_default_graph()
BATCH,T,D=22,10,64
X=tf.placeholder(dtype=tf.float32,shape=[BATCH,T,D])

with tf.variable_scope('bi',reuse=tf.AUTO_REUSE):
    ndims=[32,64]
    cells=[tf.contrib.rnn.BasicLSTMCell(d) for d in ndims]
    forward_cell=tf.contrib.rnn.MultiRNNCell(cells)
    #注意,要new一边不能用上面的cells,不然不会创建新的weight
    cells=[tf.contrib.rnn.BasicLSTMCell(d) for d in ndims]
    backward_cell=tf.contrib.rnn.MultiRNNCell(cells)
    
    output,states=tf.nn.bidirectional_dynamic_rnn(forward_cell,backward_cell,X,dtype=tf.float32,time_major=False)

In [7]:
# print(output)
print(states)
beam_states=tf.contrib.seq2seq.tile_batch(states,3)
print(beam_states)

((LSTMStateTuple(c=<tf.Tensor 'bi/bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(22, 32) dtype=float32>, h=<tf.Tensor 'bi/bidirectional_rnn/fw/fw/while/Exit_4:0' shape=(22, 32) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'bi/bidirectional_rnn/fw/fw/while/Exit_5:0' shape=(22, 64) dtype=float32>, h=<tf.Tensor 'bi/bidirectional_rnn/fw/fw/while/Exit_6:0' shape=(22, 64) dtype=float32>)), (LSTMStateTuple(c=<tf.Tensor 'bi/bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(22, 32) dtype=float32>, h=<tf.Tensor 'bi/bidirectional_rnn/bw/bw/while/Exit_4:0' shape=(22, 32) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'bi/bidirectional_rnn/bw/bw/while/Exit_5:0' shape=(22, 64) dtype=float32>, h=<tf.Tensor 'bi/bidirectional_rnn/bw/bw/while/Exit_6:0' shape=(22, 64) dtype=float32>)))
((LSTMStateTuple(c=<tf.Tensor 'tile_batch/Reshape:0' shape=(66, 32) dtype=float32>, h=<tf.Tensor 'tile_batch/Reshape_1:0' shape=(66, 32) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'tile_batch/Reshape_2:0' shape=(66, 64) dtype

In [8]:
tf.trainable_variables()

[<tf.Variable 'bi/bidirectional_rnn/fw/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(96, 128) dtype=float32_ref>,
 <tf.Variable 'bi/bidirectional_rnn/fw/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'bi/bidirectional_rnn/fw/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(96, 256) dtype=float32_ref>,
 <tf.Variable 'bi/bidirectional_rnn/fw/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'bi/bidirectional_rnn/bw/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(96, 128) dtype=float32_ref>,
 <tf.Variable 'bi/bidirectional_rnn/bw/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'bi/bidirectional_rnn/bw/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(96, 256) dtype=float32_ref>,
 <tf.Variable 'bi/bidirectional_rnn/bw/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(256,) dtype=float32_ref>]

In [39]:
##
tf.reset_default_graph()
s=tf.constant(value=1,shape=[20,5,2])
h=tf.constant(value=1,shape=[20,5,2])
print(s)
c=tf.contrib.seq2seq.tile_batch((s,h),3)
print(c)

Tensor("Const:0", shape=(20, 5, 2), dtype=int32)
(<tf.Tensor 'tile_batch/Reshape:0' shape=(60, 5, 2) dtype=int32>, <tf.Tensor 'tile_batch/Reshape_1:0' shape=(60, 5, 2) dtype=int32>)


In [18]:
#decoder
tf.reset_default_graph()
BATCH,T,D=22,10,17
X=tf.placeholder(dtype=tf.float32,shape=[BATCH,T,D])
X_len=tf.placeholder(dtype=tf.int32,shape=[BATCH])
with tf.variable_scope('decoder'):
    dims=[32,64]
    cells=[tf.contrib.rnn.BasicLSTMCell(d) for d in dims]
    decode_cell=tf.contrib.rnn.MultiRNNCell(cells)
    decode_init_state=decode_cell.zero_state(BATCH,tf.float32)
    #decoder network
    helper=tf.contrib.seq2seq.TrainingHelper(X,X_len)
    print(helper)
    mydecoder=tf.contrib.seq2seq.BasicDecoder(decode_cell,helper,decode_init_state)
    print(mydecoder)
    final_outputs, final_state, final_sequence_lengths=tf.contrib.seq2seq.dynamic_decode(mydecoder)

In [19]:
print(final_outputs)

BasicDecoderOutput(rnn_output=<tf.Tensor 'decoder/decoder/transpose:0' shape=(22, ?, 64) dtype=float32>, sample_id=<tf.Tensor 'decoder/decoder/transpose_1:0' shape=(22, ?) dtype=int32>)


In [20]:
print(final_state)

(LSTMStateTuple(c=<tf.Tensor 'decoder/decoder/while/Exit_3:0' shape=(22, 32) dtype=float32>, h=<tf.Tensor 'decoder/decoder/while/Exit_4:0' shape=(22, 32) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'decoder/decoder/while/Exit_5:0' shape=(22, 64) dtype=float32>, h=<tf.Tensor 'decoder/decoder/while/Exit_6:0' shape=(22, 64) dtype=float32>))


In [21]:
print(final_sequence_lengths)

Tensor("decoder/decoder/while/Exit_9:0", shape=(22,), dtype=int32)


In [15]:
#inference,sample and greed
tf.reset_default_graph()
BATCH,D=22,17
X=tf.placeholder(dtype=tf.float32,shape=[BATCH,D])
X_len=tf.placeholder(dtype=tf.int32,shape=[BATCH])

embeding=tf.constant(dtype=tf.float32,shape=[10000,67],value=1.0)
sos=tf.constant(0,shape=[BATCH])
eos=tf.constant(1,shape=[])

with tf.variable_scope('decoder'):
    dims=[32,64]
    cells=[tf.contrib.rnn.BasicLSTMCell(d) for d in dims]
    decode_cell=tf.contrib.rnn.MultiRNNCell(cells)
    decode_init_state=decode_cell.zero_state(BATCH,tf.float32)
    
#     helper=tf.contrib.seq2seq.GreedyEmbeddingHelper(embeding,sos,eos)
    helper=tf.contrib.seq2seq.SampleEmbeddingHelper(embeding,sos,eos,softmax_temperature=1.0)
    decoder=tf.contrib.seq2seq.BasicDecoder(decode_cell,helper,decode_init_state)
    final_output,final_state,final_len=tf.contrib.seq2seq.dynamic_decode(decoder,maximum_iterations=20)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [16]:
print(final_output)

BasicDecoderOutput(rnn_output=<tf.Tensor 'decoder/decoder/transpose:0' shape=(22, ?, 64) dtype=float32>, sample_id=<tf.Tensor 'decoder/decoder/transpose_1:0' shape=(22, ?) dtype=int32>)


In [17]:
#inference beam search!
tf.reset_default_graph()
BATCH,D=22,17
BW=3
X=tf.placeholder(dtype=tf.float32,shape=[BATCH,D])
X_len=tf.placeholder(dtype=tf.int32,shape=[BATCH])
sos=tf.constant(0,shape=[BATCH])
eos=tf.constant(1,shape=[])
embeding=tf.constant(dtype=tf.float32,shape=[10000,67],value=1.0)

with tf.variable_scope('decoder'):
    dims=[32,64]
    cells=[tf.contrib.rnn.BasicLSTMCell(d) for d in dims]
    decode_cell=tf.contrib.rnn.MultiRNNCell(cells)
    decode_init_state=decode_cell.zero_state(BATCH,tf.float32)
    decode_init_state=tf.contrib.seq2seq.tile_batch(decode_init_state,BW)
    decoder=tf.contrib.seq2seq.BeamSearchDecoder(decode_cell,embeding,sos,eos,decode_init_state,BW)
    final_output,final_state,final_len=tf.contrib.seq2seq.dynamic_decode(decoder,maximum_iterations=20)

In [18]:
print(final_len)
print(final_state)

Tensor("decoder/decoder/while/Exit_14:0", shape=(22, 3), dtype=int32)
BeamSearchDecoderState(cell_state=(LSTMStateTuple(c=<tf.Tensor 'decoder/decoder/while/Exit_5:0' shape=(22, 3, 32) dtype=float32>, h=<tf.Tensor 'decoder/decoder/while/Exit_6:0' shape=(22, 3, 32) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'decoder/decoder/while/Exit_7:0' shape=(22, 3, 64) dtype=float32>, h=<tf.Tensor 'decoder/decoder/while/Exit_8:0' shape=(22, 3, 64) dtype=float32>)), log_probs=<tf.Tensor 'decoder/decoder/while/Exit_9:0' shape=(22, 3) dtype=float32>, finished=<tf.Tensor 'decoder/decoder/while/Exit_10:0' shape=(22, 3) dtype=bool>, lengths=<tf.Tensor 'decoder/decoder/while/Exit_11:0' shape=(22, 3) dtype=int64>, accumulated_attention_probs=())


In [55]:
print(final_output)

FinalBeamSearchDecoderOutput(predicted_ids=<tf.Tensor 'decoder/decoder/transpose:0' shape=(22, ?, 3) dtype=int32>, beam_search_decoder_output=BeamSearchDecoderOutput(scores=<tf.Tensor 'decoder/decoder/transpose_1:0' shape=(22, ?, 3) dtype=float32>, predicted_ids=<tf.Tensor 'decoder/decoder/transpose_2:0' shape=(22, ?, 3) dtype=int32>, parent_ids=<tf.Tensor 'decoder/decoder/transpose_3:0' shape=(22, ?, 3) dtype=int32>))


In [59]:
final_state.cell_state

(LSTMStateTuple(c=<tf.Tensor 'decoder/decoder/while/Exit_5:0' shape=(22, 3, 32) dtype=float32>, h=<tf.Tensor 'decoder/decoder/while/Exit_6:0' shape=(22, 3, 32) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'decoder/decoder/while/Exit_7:0' shape=(22, 3, 64) dtype=float32>, h=<tf.Tensor 'decoder/decoder/while/Exit_8:0' shape=(22, 3, 64) dtype=float32>))

In [66]:
final_output.predicted_ids.shape[0].value

22

In [ ]:
tf.sequence_mask()

In [40]:
#graph

def common_network(X):
    with tf.variable_scope('mynn',reuse=tf.AUTO_REUSE):
        W=tf.get_variable('W',shape=(),dtype=tf.float32,initializer=tf.constant_initializer(3.0))
    return X*W
def train_net():
    X=tf.constant(12.0,name='train_X')
    return common_network(X)
def infer_net():
    X=tf.constant(10.0,name='infer_X')
    return common_network(X)

In [41]:
tf.reset_default_graph()
g1=tf.Graph()
with g1.as_default():
    Y_train=train_net()
    print(tf.trainable_variables())
    sess1=tf.Session()
    sess1.run(tf.global_variables_initializer())
    print(sess1.run(Y_train))
g2=tf.Graph()
with g2.as_default():
    Y_infer=infer_net()
    print(tf.trainable_variables())
    sess2=tf.Session()
    sess2.run(tf.global_variables_initializer())
    print(sess2.run(Y_infer))
# Y_infer=infer_net()

# sess1=tf.Session(graph=g1)
# # sess2=tf.Session()
# sess1.run(tf.global_variables_initializer())
# print(sess1.run(Y_train))
# print(sess1.run(Y_infer))

# sess2.run(tf.global_variables_initializer())
# print(sess2.run(Y_train))
# print(sess2.run(Y_infer))

[<tf.Variable 'mynn/W:0' shape=() dtype=float32_ref>]
36.0
[<tf.Variable 'mynn/W:0' shape=() dtype=float32_ref>]
30.0


In [28]:
#attention
tf.reset_default_graph()
BATCH,T=22,15
T1=11
embed_size=122
#这里没有定义encoder,假设encoder的输出是encoder_size,
encoder_size=9
memory=tf.constant(1.0,shape=[BATCH,T1,encoder_size],dtype=tf.float32)
X=tf.placeholder(dtype=tf.float32,shape=[BATCH,T,embed_size])
X_len=tf.placeholder(dtype=tf.int32,shape=[BATCH])
dims=[128,256]
cell_list=[tf.contrib.rnn.BasicRNNCell(d) for d in dims]
cells=tf.contrib.rnn.MultiRNNCell(cell_list,state_is_tuple=True)

attention_depth=256
attention_mechanism=tf.contrib.seq2seq.LuongAttention(attention_depth,memory,dtype=tf.float32)

# alignments_size:序列的长度
# batch_size
# keys:(batch,alignments_size,attention_depth),
#           Tensor("LuongAttention/memory_layer/Tensordot:0", shape=(22, 10, 256), dtype=float32)
# memory_layer:把memory,hs经过memory_layer转化成和ht相同的维度后,计算score,ht * W * hs
# state_size:T
#values:传入的memory
# print(attention_mechanism.values)
# init_atten=attention_mechanism.initial_state(BATCH,tf.float32)
# query=tf.constant(1.0,shape=[BATCH,attention_depth])
# attention_mechanism(query,init_atten)
cells=tf.contrib.seq2seq.AttentionWrapper(cells,attention_mechanism,
                                    attention_layer_size=None,
                                    alignment_history=True,
                                    output_attention=True)
print(cells)
initial_state=cells.zero_state(BATCH,tf.float32)
# # print(initial_state)
helper=tf.contrib.seq2seq.TrainingHelper(X,X_len)
decoder=tf.contrib.seq2seq.BasicDecoder(cells,helper,initial_state)
rnn_output=tf.contrib.seq2seq.dynamic_decode(decoder)

In [29]:
print(rnn_output[0])
print(rnn_output[1])
# for  v in tf.trainable_variables():
#     print(v)

BasicDecoderOutput(rnn_output=<tf.Tensor 'decoder/transpose:0' shape=(22, ?, 9) dtype=float32>, sample_id=<tf.Tensor 'decoder/transpose_1:0' shape=(22, ?) dtype=int32>)
AttentionWrapperState(cell_state=(<tf.Tensor 'decoder/while/Exit_3:0' shape=(22, 128) dtype=float32>, <tf.Tensor 'decoder/while/Exit_4:0' shape=(22, 256) dtype=float32>), attention=<tf.Tensor 'decoder/while/Exit_5:0' shape=(22, 9) dtype=float32>, time=<tf.Tensor 'decoder/while/Exit_6:0' shape=() dtype=int32>, alignments=<tf.Tensor 'decoder/while/Exit_7:0' shape=(22, 11) dtype=float32>, alignment_history=<tensorflow.python.ops.tensor_array_ops.TensorArray object at 0x7fdea86cbac8>, attention_state=<tf.Tensor 'decoder/while/Exit_9:0' shape=(22, 11) dtype=float32>)


In [31]:
attention_images = (rnn_output[1].alignment_history.stack())
print(attention_images)

Tensor("TensorArrayStack_1/TensorArrayGatherV3:0", shape=(?, 22, 11), dtype=float32)


In [124]:
print(rnn_output[1])

AttentionWrapperState(cell_state=(<tf.Tensor 'decoder/while/Exit_3:0' shape=(22, 128) dtype=float32>, <tf.Tensor 'decoder/while/Exit_4:0' shape=(22, 256) dtype=float32>), attention=<tf.Tensor 'decoder/while/Exit_5:0' shape=(22, 256) dtype=float32>, time=<tf.Tensor 'decoder/while/Exit_6:0' shape=() dtype=int32>, alignments=<tf.Tensor 'decoder/while/Exit_7:0' shape=(22, 10) dtype=float32>, alignment_history=(), attention_state=<tf.Tensor 'decoder/while/Exit_8:0' shape=(22, 10) dtype=float32>)


In [2]:
#tf.contrib.framework.nest.map_structure

In [25]:
N,D1,D2,D3=128,31,64,96
X1=tf.constant(value=1.0,shape=[N,D1])
X2=tf.constant(value=1.0,shape=[N,D2])
X3=tf.constant(value=1.0,shape=[N,D3])
def func(X1,X2,X3):
    D1=X1.get_shape().as_list()[-1]
    D2=X2.get_shape().as_list()[-1]
    return tf.split(X2,[D1,D2-D1],axis=-1)
NEWX2,_=tf.contrib.framework.nest.map_structure(func,X1,X2,X3)
NEWX2.get_shape().assert_is_compatible_with(X1.get_shape())
tf.contrib.framework.nest.assert_same_structure(X1,X1)

In [30]:
tf.contrib.framework.nest.assert_same_structure(X1,X3)

In [28]:
print(ss)

None
